In [1]:
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

In [2]:
server_params = StdioServerParameters(
    command='python',
    args=['-m', 'mcp_this'],
)

async with stdio_client(server_params) as (read, write):  # noqa: SIM117
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()
        print("Available tools:\n-----------------------\n")
        for tool in tools.tools:
            print(f"- `{tool.name}`")

        dir_tree_result = await session.call_tool(
            'get-directory-tree',
            {'directory': '..'},
        )
        find_files_results = await session.call_tool(
            'find-files',
            {
                'directory': '/Users/shanekercheval/repos/mcp-this/src/mcp_this',
                'arguments': '-name *.py',
            },
        )
        find_text_patterns_result = await session.call_tool(
            'find-text-patterns',
            {
                'directory': '/Users/shanekercheval/repos/mcp-this/src/mcp_this',
                'arguments': '--include=*.py',
                'pattern': 'def ',
            },
        )
        extract_file_text_result = await session.call_tool(
            'extract-file-text',
            {
                'file': '/Users/shanekercheval/repos/mcp-this/src/mcp_this/mcp_server.py',
                'arguments': '| head -20',
            },
        )
        code_info_result = await session.call_tool(
            'extract-code-info',
            {
                'files': '/Users/shanekercheval/repos/mcp-this/src/mcp_this/*.py',
                'types': 'functions,classes',
            },
        )

Available tools:
-----------------------

- `get-directory-tree`
- `find-files`
- `find-text-patterns`
- `extract-file-text`
- `extract-code-info`
- `web-scraper`


In [3]:
dir_tree_result

CallToolResult(meta=None, content=[TextContent(type='text', text='..\n├── .github\n│\xa0\xa0 └── workflows\n│\xa0\xa0     └── tests.yaml\n├── .gitignore\n├── .ruff.toml\n├── LICENSE\n├── Makefile\n├── README.md\n├── README_PRESETS.md\n├── documentation\n│\xa0\xa0 └── images\n│\xa0\xa0     ├── custom-tool-example.png\n│\xa0\xa0     ├── custom-tool.png\n│\xa0\xa0     ├── default-tools.png\n│\xa0\xa0     ├── server-default.png\n│\xa0\xa0     ├── servers-default-custom.png\n│\xa0\xa0     └── tool-allow.png\n├── examples\n│\xa0\xa0 ├── cli.py\n│\xa0\xa0 ├── custom_tools.yaml\n│\xa0\xa0 ├── examples-default-tools.ipynb\n│\xa0\xa0 ├── examples-editing-tools.ipynb\n│\xa0\xa0 ├── examples-github-tools.ipynb\n│\xa0\xa0 ├── examples-passing-tools.ipynb\n│\xa0\xa0 ├── mcp_config_cli.json\n│\xa0\xa0 ├── prompts_example.yaml\n│\xa0\xa0 ├── temp\n│\xa0\xa0 │\xa0\xa0 └── nested\n│\xa0\xa0 │\xa0\xa0     └── example\n│\xa0\xa0 │\xa0\xa0         └── directory\n│\xa0\xa0 └── tools_example.yaml\n├── pyproj

---

In [4]:
print(dir_tree_result.content[0].text)

..
├── .github
│   └── workflows
│       └── tests.yaml
├── .gitignore
├── .ruff.toml
├── LICENSE
├── Makefile
├── README.md
├── README_PRESETS.md
├── documentation
│   └── images
│       ├── custom-tool-example.png
│       ├── custom-tool.png
│       ├── default-tools.png
│       ├── server-default.png
│       ├── servers-default-custom.png
│       └── tool-allow.png
├── examples
│   ├── cli.py
│   ├── custom_tools.yaml
│   ├── examples-default-tools.ipynb
│   ├── examples-editing-tools.ipynb
│   ├── examples-github-tools.ipynb
│   ├── examples-passing-tools.ipynb
│   ├── mcp_config_cli.json
│   ├── prompts_example.yaml
│   ├── temp
│   │   └── nested
│   │       └── example
│   │           └── directory
│   └── tools_example.yaml
├── pyproject.toml
├── src
│   └── mcp_this
│       ├── __init__.py
│       ├── __main__.py
│       ├── configs
│       │   ├── default.yaml
│       │   ├── editing.yaml
│       │   └── github.yaml
│       ├── mcp_server.py
│       ├── prompts.py
│       ├──

---

In [5]:
print(find_files_results.content[0].text)

./__init__.py
./__main__.py
./mcp_server.py
./prompts.py
./tools.py



---

In [6]:
print(find_text_patterns_result.content[0].text)

./tools.py:34:    def get_full_description(self) -> str:
./tools.py:70:def build_command(command_template: str, parameters: dict[str, str]) -> str:
./tools.py:117:async def execute_command(cmd: str) -> str:
./tools.py:168:def parse_tools(config: dict) -> list[ToolInfo]:
./tools.py:193:def create_tool_info(tool_name: str, tool_config: dict) -> ToolInfo:
./tools.py:238:        f"async def {function_name}({param_string}) -> str:",
./prompts.py:30:def validate_prompt_config(prompt_name: str, prompt_config: dict) -> None:
./prompts.py:65:def validate_prompt_argument_config(prompt_name: str, arg_name: str, arg_config: dict) -> None:
./prompts.py:109:def parse_prompts(config: dict) -> list[PromptInfo]:
./__main__.py:17:def find_default_config() -> str | None:
./__main__.py:36:def get_preset_config(preset_name: str) -> str | None:
./__main__.py:52:def main() -> None:
./mcp_server.py:22:def render_template(template: str, kwargs: dict) -> str:
./mcp_server.py:37:    def handle_if_block(match: re

---

In [7]:
print(extract_file_text_result.content[0].text)

     1	#!/usr/bin/env python3
     2	"""
     3	MCP Server that dynamically creates command-line tools based on a YAML configuration file.
     4	
     5	Each tool maps to a command-line command that can be executed by the server.
     6	"""
     7	import os
     8	import yaml
     9	import json
    10	import re
    11	from pathlib import Path
    12	from mcp.server.fastmcp import FastMCP
    13	import sys
    14	from collections.abc import Callable
    15	from mcp_this.tools import ToolInfo, build_command, execute_command, parse_tools
    16	from mcp_this.prompts import PromptInfo, parse_prompts
    17	
    18	
    19	mcp = FastMCP("Dynamic CLI Tools")
    20	



---

In [8]:
print(code_info_result.content[0].text)

=== File: /Users/shanekercheval/repos/mcp-this/src/mcp_this/__init__.py ===
--- functions ---
No function definitions found
--- classes ---
No classes found

=== File: /Users/shanekercheval/repos/mcp-this/src/mcp_this/__main__.py ===
--- functions ---
17:def find_default_config() -> str | None:
36:def get_preset_config(preset_name: str) -> str | None:
52:def main() -> None:
--- classes ---
No classes found

=== File: /Users/shanekercheval/repos/mcp-this/src/mcp_this/mcp_server.py ===
--- functions ---
22:def render_template(template: str, kwargs: dict) -> str:
37:    def handle_if_block(match: re.Match) -> str:
58:def get_default_config_path() -> Path | None:
69:def load_config(config_path: str | None = None, tools: str | None = None) -> dict:
128:def validate_config(config: dict) -> None:
163:def validate_tool_config(tool_id: str, tool_config: dict) -> None:
187:def register_parsed_tools(tools_info: list[ToolInfo]) -> None:
216:def register_prompts(prompts_info: list[PromptInfo]) -> N